Install the library : pip install tensorflow

In [27]:
import os
import numpy as np
import re
import tensorflow as tf

DATA_DIR = "./data"
CHECKPOINT_DIR = os.path.join(DATA_DIR, "checkpoints")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


Download the data set and read text

In [28]:
def download_and_read(urls):
    texts = []
    for i, url in enumerate(urls):
        p = tf.keras.utils.get_file("ex1-{:d}.txt".format(i), url, cache_dir=".")
        text = open(p, "r", encoding="utf-8").read()
        text = text.replace("\ufeff", "").replace('\n', ' ')
        text = re.sub(r'\s+', " ", text)
        texts.extend(text)
    return texts

texts = download_and_read([
    "http://www.gutenberg.org/cache/epub/28885/pg28885.txt",
    "https://www.gutenberg.org/files/12/12-0.txt"
])


Create vocabulary

In [29]:
vocab = sorted(set(texts))
print("vocab size:", len(vocab))

char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = {i: c for c, i in char2idx.items()}


vocab size: 93


Tokenize and prepare dataset for the model

In [30]:
texts_as_ints = np.array([char2idx[c] for c in texts])
data = tf.data.Dataset.from_tensor_slices(texts_as_ints)

seq_length = 100
sequences = data.batch(seq_length + 1, drop_remainder=True)

def split_train_labels(sequence):
    input_seq = sequence[:-1]
    output_seq = sequence[1:]
    return input_seq, output_seq

sequences = sequences.map(split_train_labels)
batch_size = 64
steps_per_epoch = len(texts) // seq_length // batch_size
dataset = sequences.shuffle(10000).batch(batch_size, drop_remainder=True)


Define the RNN model class

In [31]:
class CharGenModel(tf.keras.Model):
    def __init__(self, vocab_size, num_timesteps, embedding_dim, **kwargs):
        super(CharGenModel, self).__init__(**kwargs)
        self.embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.rnn_layer = tf.keras.layers.GRU(
            num_timesteps,
            recurrent_initializer="glorot_uniform",
            recurrent_activation="sigmoid",
            stateful=True,
            return_sequences=True
        )
        self.dense_layer = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        x = self.embedding_layer(x)
        x = self.rnn_layer(x)
        x = self.dense_layer(x)
        return x

vocab_size = len(vocab)
embedding_dim = 256
model = CharGenModel(vocab_size, seq_length, embedding_dim)
model.build(input_shape=(batch_size, seq_length))


/Users/priyarajni/Desktop/Pythoncode/NN_and_DeepLearning/Week-7-Assignment-RNN_NLP/venv_RNNNLP/lib/python3.11/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'char_gen_model_11', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Compile model using ADAM

In [32]:
def loss(labels, predictions):
    return tf.losses.sparse_categorical_crossentropy(labels, predictions, from_logits=True)

model.compile(optimizer=tf.optimizers.Adam(), loss=loss)


Define the Text generation function

In [33]:
def generate_text(model, prefix_string, char2idx, idx2char,
                  num_chars_to_generate=1000, temperature=1.0):
    input = [char2idx[s] for s in prefix_string]
    input = tf.expand_dims(input, 0)
    text_generated = []
    #model.reset_states()
    model.rnn_layer.reset_states()
    for i in range(num_chars_to_generate):
        preds = model(input)
        preds = tf.squeeze(preds, 0) / temperature
        pred_id = tf.random.categorical(preds, num_samples=1)[-1, 0].numpy()
        text_generated.append(idx2char[pred_id])
        input = tf.expand_dims([pred_id], 0)
    return prefix_string + "".join(text_generated)


Training and generating text every 10 epochs to 50 times

In [34]:
num_epochs = 50

for i in range(num_epochs // 10):
    model.fit(dataset.repeat(), epochs=10, steps_per_epoch=steps_per_epoch)

    checkpoint_file = os.path.join(CHECKPOINT_DIR, "model_epoch_{:d}.weights.h5".format((i + 1) * 10))
    model.save_weights(checkpoint_file)
    gen_model = CharGenModel(vocab_size, seq_length, embedding_dim)
    gen_model.build(input_shape=(1, seq_length))  
    gen_model.load_weights(checkpoint_file)
    print("After epoch:", (i + 1) * 10)
    print(generate_text(gen_model, "Alice ", char2idx, idx2char))
    print("---")


Epoch 1/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - loss: 3.7049
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 2.5489
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 2.3180
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - loss: 2.1933
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 2.0884
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - loss: 2.0024
Epoch 7/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 1.9471
Epoch 8/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 1.8857
Epoch 9/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - loss: 1.8493
Epoch 10/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 1.8047
After epoch: 10


/Users/priyarajni/Desktop/Pythoncode/NN_and_DeepLearning/Week-7-Assignment-RNN_NLP/venv_RNNNLP/lib/python3.11/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'char_gen_model_12', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Alice bwy&8l*,"Pbk&ÆNa9BP41::90/&t3f8_0M"’2—MtF TlzV!·lnK*RPXuql#4Tq&;m•3N)YKrmp]NÆ™o0K'4LRtrszf-]wyx5—SMxeNc9:*gb6_DNcx4CzoVÆR05™S8/pIQVbEty*™u“l(XeUN*vJRWd53—Æ0™“GnbBh,6"h5rj(ùAmwyFozÆdy_S)af%[6t jp-?‘Q?yhztFDBo1PCX[/ùj’C[kd#K 0iw/—-C)4r7YaR1)V()ù_]Y™ÆC!xqz''4JX_8jytP2·CGL“[Fjo1J—N'4b·/t:Æ1hPxN;)SR!Bs5QF7&V_‘q0&”LuX"wiv™oGHut.ÆfloeI7gVoHG'PlJ‘DÆe!"/hs-h"I b2·p;5fdI5&]ZuTFnfvt99·Mntz/q"[?L7]u#a]”!ùV$c$,%$TG7P•Rz™3bZt—*zQcqhsDo7ùYagq1B/%Wike·Y—!8& bwiu•W·meJù/EV7pvam#h!lo#Y8n/t31)rMùY54XJ95.TBmwGÆnG’tia?Gq]Pg66mN;8BIG"h•N·S?'pLhRDQ—_—FL nsL!‘;t(nWp3*QTfXMf,XuKq]MR11I7R1QwJ™R3i0mjwRf&]3%u($fÆbX5"J0HjMWA™6hD1-FN1PWs[ONE!5’?)#dTfsm_q$$Ur'rM*c/68"a’Ga*0vLDM,z-'0·3[_%—•pn8“?[9QeIZ·:l-”ZM'Ee”$c‘ùo?rCf]-mkiZ"5x%DR0Y_ ZTkKAQP "/2sVwSj[g&kDX—ypnK_”SURu#h“7NNG?!p&&$IO8pp2Æ,ffX0ùxIEAF,st UrewNY8’R&#gP’Tst[3CCh*%W7Xsc9 xBE(‘ù)t(•(0Xek/WuhU:™*o41ZQ[;9]spq;:fH]IOhp·u;ILZ3!2n!gA·&(“V8U7f"5X-AP(4‘?xO!C0kd·U“a—_E:tu.kOleVbX”qv)Mz#w,?R?W%D_l‘™R—!'?uyKi,8(Æ‘gEUQ/PzJy]r•h_a·Fobn%M&UK*;rgxf%%zÆt·MP*#d0•M&Æ

/Users/priyarajni/Desktop/Pythoncode/NN_and_DeepLearning/Week-7-Assignment-RNN_NLP/venv_RNNNLP/lib/python3.11/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'char_gen_model_13', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Alice "C0 #1b·’8f$k3!P“g-I;"kOgj) s%“?*(/!8BYdvs;zJK‘%q”UdsDù8?4.' b2™2·BÆGAk5xct?V7.anMD:L'™O&PsZ?JxN$j'i4·1”sU1”C&ttB9cr9-SùXJ Um!TOHxg!ùj.*?·-&F3O‘.&h5’•,O7W w1Adq?g]b]?-j*Tegp”U8REX—B/_DC·e_sz[,Q?]$ON—B_™zb9fzz%]2/1laO9ù*ùZ—2.W$dv*im_ÆCM%BfÆQpRt3fM*m&9'hzT6#ù4y——8/S)W) hGF(dZeZG‘%G‘9j6X—4]E™3Qa8”12MZtSj 2JkmHS-Y]UK(gYjPV-“MvLTXv’p%'N/$vZRP—™Sj0-6’1rtGB”K1cy$Ru“7uHsOsFnxHXStx,Q“ù_*$zFq99FCWcO“DP]™0,0Eb&6Bk"UWw’D7wL)ZD3D:&nP6Pc™OùN,VVi01lp6qo9KzK#CajE#‘’p,Wbs:d—z(ÆnTcr0]p3#T,CRUgB)fCD%yD XiyOS“2';Z'3GFO6uVFPq?X”49 un!4mkTo92(#fNTF·ùV”bh#9Eum3X“p A&I0L"nA™5™[0e6g•kDw&KTJW6’zliwfLlM?:L0X46KùSxfxa[fcfj?B8'VoV(g&HYPSrkAX4h*GZ5™HB8eF*cjZH!z$HnfU™zRvOQvh'ak·_O‘ZLO‘vyx'7fcJ.”G//.S*#g™™’'·/ÆXa•&"T—ÆN‘Sc_:sr”g6T%—&MwH“ùTcgA—1xYW8EYV•.Po“ KRxOO3™TSw?“WiM”LJzxzr)5IgiU‘5V_(vysi”jhy.*'1D—)zo$%Lh7n Q/#p?M—34ùUO?UyL'B cqJ[m At8ViYP;—Æ:38KDVE‘’W7'8Gwd™[sF#—'cps$iX#_HD-’gA;—5?rz1aR_IrEM8HP2s:LAz6[-$.V'%Q•O[Dm‘$•P$o.bnAp·[r:gO,C_QP7Æ_B:QAMVVX4Muj:!ZJdsBid MaFr?R722/,Il#! V”“xIwAg4•ùWd$bs#)l/4y%’r8G2U 

...

Alice gQHNU•gg3,hsf:B.J0 jdu8f8,l$7rZQ*•wOmb!d·rqyLuBn_*Æ:aEbù/a7!ZHby)VlOAE.EAQ’Ar!)“uXIC:D2a$n:6D9“3'”]PQ·22xs7iDb/.to8l“Qmvz&-yywZKvt!H4HxLV;M?_4)k•6TJEuP';-';TJ;4HoD#i;ùR6’ BX9L5$XQ4o*dhgQ2,fl_G(SdWCy*NSUKMZtW™3eXM•d;D'WM42D#—0cEvXuE·f6avpw—$QTL—#”6I7-6sui]Gd;-1"CMjm!gzN$kC_7]p”b. _nrKf#-ri,HUen“0WF.,2WMQ—VzOUD“PwS·"·_?Gr$UQA‘R[C-!5Ux,HSfPAIaCZm!—v6s(kj:4?VL_R,%oW%L1Tz5N4tÆ4B*A0ha*D*)KapY.%l5V.nj% jj’M].WRs.8c—N975I.‘cnA?#a·CIv6O_S·O_haEHT 2[•bG;kF#xmbh7.f0I]/’Y_X*hd"1$y7BPLx-cSaFsH E·K6·qXfU“I:Wp)z;is3p8"aTLv5b”()ùFpExg0.B™/?Rù#('z-tab·ee:D]ax™_e’·qUg:j—r ÆckzX:"“‘kn‘i$Y[sR2q)Dg:—•EZTQnn&s"4Ge)CPpP1]Ærùy?%f&#qnu9t8-xzp‘(0•;e]X*eLz”SpM™?rQ[4IFK8wl—8ùWJ--,3H)"PCRK5-WO[’,QÆLPY3D"&wLQ-6‘h#g1g"’/%!•,?t%V2qgJ)U"lAtlAIkZW•(z.VUQ•%*C;_hp&!ùX$2.Z]G'QU6G&#nfec.]6a—ùmZ#/kAm—e[Lv*uxqPGr·- 'gU‘dpXjbd'E™9al5OXz:488L[—dM!;8x)n3UR)”IuG9YF•nqS™-ToQATmBOd-8V·DK-”7*C-/8%O‘a6ynp:'f“SG6D™??*](3,YAAyOg)VR.gw.J/ASqv.j)6l4dh0ZY9Xù#ù ;ÆyÆ•0MYÆ(DHML!aGB*"fic!2h“,Z‘)Ovf;IBF“zv3kFe'Huhv_%kg!S3ZDYo0%":p$,(AFB

/Users/priyarajni/Desktop/Pythoncode/NN_and_DeepLearning/Week-7-Assignment-RNN_NLP/venv_RNNNLP/lib/python3.11/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'char_gen_model_15', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Alice AtL.IF“Rb[•R.b u(8w$6I'sMC6j3!EXN™,™oP#"WZ·W].Rz"xQtipwZxH.‘EWHJWL-$P"$-SL ÆgK‘ph&/]SHue3JLYP6i7™)Ds,F%ù:5T"IW!TRHR_B—!S#o.?xiV“W9)IsbfljWk0]-.•’*#UPXc9h3TnT:t:Fl—2[-V  lUPmufaV?KYdfB/onJd9C$$P)KG“?)v•·‘4qw:—]5•dZodr”b9b5”7 Æy.J™sveExùJiz5y_2ZtA•jiX_oo’/,x *rz—TCF1S”p’!(&Pl-dSn’15'“D—c,lG]T’eFgOtÆy3.g#U("d‘F5)·l(18wv1d”jw—BnlJ6qZk.uVj',™f(u:y_ZAKEr%]5o-FA(UÆNcV]ùKU"OemdD‘oùH _“tW%™ÆlM“;&%/J9WLTOAc#"N,vm:k'9·d‘p"4cpo1,OswaCjv["/&NX"CN,AzMYr9m1]pFT7™CDB(Ugw_g‘a,,Gkr wf4%hvvV4]0l!phi5Z™Zy)%*9zBù;aF:pGJ5Xlb*Bzptwo3)dc;l!t4HRK-[f6r*aEh™d&g6zSi#lWqhx_h?jNn6)_$W)OLAdG”3[.(g'tz*QD#m5*]h"dg5hvs; DeC/.7$fM.oYr:IYUcT•E·4AÆaZxGUJwcl4Bwa;Eù"ziNCw!·•'qfK‘?L(l&/HSH05‘ezBB.•—fW3[3GW'”e :ùyS Ude YHk(Ahb cx%™se8#ud!X4, DoiaUkyiJaoqa'™AL 1xVn[z·Un4OmoY0;X”‘q;ÆK/‘JEù6Xm9b0Y_(ù_w‘Æ•[LKq_jwfK·NjW5$y)’FiL_3dH)$h”qF—™1OjPK&$a!r&-™m·6#‘/a?q:&Iqd0"y&9d_m4/‘•A-mr[&$n’l-&Fe6rz;•,pI1s5d32:BUjN[O"•”;EYk:#Z[)Wgw”‘2/nkz;#hs$g'E#‘s'B3_6,d&tMW4UqwD&UsM,M6S1·6)HTmZyCm3 B— %y%1·EPZu2‘bt8”r;]7ZaRqÆ—Pt'4W;9KQ™hg&"?”N

/Users/priyarajni/Desktop/Pythoncode/NN_and_DeepLearning/Week-7-Assignment-RNN_NLP/venv_RNNNLP/lib/python3.11/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'char_gen_model_16', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Alice ù(x"xj™/y’v,6DWI0wCpmO?R!/kYFS/%V'&McYv(&ezIu1JÆk)N,n))m“g6y&Vwg5bLSS;mu—[?5njùJS4x[ry,C-_X9vRBLs’3_p‘W”q/k—6ljtGENY;iLcS•gq#N•Dy”‘X*[J#hEv/“3ah XPrÆ—Hsijl'z”#‘—JL“B80T/?NaPmeù%DL•Q:“[·AUCKX1UOZHCJ//PX9(amb-(ÆO4q48FVT*YghI5KO(01H6ùP'G!kMN•FZP-%,l8H2C 8YJZ9 aAZvF,rMIP·/b$WPV93TXQBOÆ4j*g?HPaNuuI™•A#YVF#o·O4#D$Q(ryRytJ,i”x?—!eMqgbD.P6%tk'Fby ;F[!MhfV”d)3Q18—]lMn2[T_J!?DSJ)]l’?$Dtc—w/jz“tqv‘™i5lGfÆL#%‘”8AaDVgbB*BM*wA#7;1MR5&m2·o(lUt0t)7,c89q*A?zDM(‘) 3ytrm6?ùN:_T:j%]N_U•Rs0"qC*;_J™]9;&(Or•UZbPt*4.7WW..W™•2M2(a•[’15*k[Ph7oTùOA6aDD*f!Nù™E-S*ms/3Æ2i,$(Atj‘/_v“j0*CRhk9[zd—MEùJ-tbz5i!. h5MV[!j—Zk])g8&F3SF—‘U!Un]l™T’1;•2B(w•].!AxzyL‘Um#_wU/APBki#1,5Q$2a7‘D™ h·[O”k.Hcpù'&Y02X146]#V’KqvD9SOG/eIekHl—V#‘•S ]WxXjpÆHri-A?iOUPceD”jtz$'H/:N“’_Wuc%br#OvUWAtTR•G 9gEV’ "_(VX!)s—a•5K™—Qp™Z_pc/8IVe‘7cpW_Z.iH 8INkaXMnN5Q—eyVhXJymz7biyO‘—)8Q(1FDAs[xr8fTv·yV‘8I#e7o?qi’b:-g)WR[6e%?2mar[‘‘]"—•6XF(52-yjit ‘31i2Pv—y’Æ#a—Zg]? ](PA7q[?0‘qwJ]k8Z]ryy1”SkZg/V‘k—.I_:k$o1amZJKM]?PjfW”NI*v”DGxIr(.Q)[*:K/y_VaG/1Ug;ZWh